 # Analisis EAR 

In [5]:
import cv2
import numpy as np
import dlib
import matplotlib.pyplot as plt
import os

ModuleNotFoundError: No module named 'dlib'

In [24]:
# Fungsi untuk menghitung Eye Aspect Ratio (EAR)
def eye_aspect_ratio(eye):
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# Fungsi untuk mendeteksi wajah dan landmark
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Folder tempat video disimpan
video_folder = "D:\TA-main\data"

# Daftar video dalam folder
video_files = [f for f in os.listdir(video_folder) if f.endswith('.avi')]

ear_values_all = []  # List untuk menyimpan nilai EAR dari semua video

for video_file in video_files:
    video_path = os.path.join(video_folder, video_file)
    cap = cv2.VideoCapture(video_path)

    ear_values = []  # List untuk menyimpan nilai EAR dari setiap frame

    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # Ubah ke grayscale untuk deteksi
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Deteksi wajah
        faces = detector(gray)

        for face in faces:
            landmarks = predictor(gray, face)

            # Ambil koordinat mata
            left_eye = []
            right_eye = []
            for n in range(36, 42):
                x = landmarks.part(n).x
                y = landmarks.part(n).y
                left_eye.append((x, y))
            for n in range(42, 48):
                x = landmarks.part(n).x
                y = landmarks.part(n).y
                right_eye.append((x, y))

            # Hitung EAR untuk setiap mata
            left_ear = eye_aspect_ratio(np.array(left_eye))
            right_ear = eye_aspect_ratio(np.array(right_eye))
            ear = (left_ear + right_ear) / 2
            ear_values.append(ear)
            
            # Gambar garis di sekitar mata (opsional, hanya untuk tujuan visualisasi)
            cv2.polylines(frame, [np.array(left_eye)], True, (0, 255, 0), 1)
            cv2.polylines(frame, [np.array(right_eye)], True, (0, 255, 0), 1)

        # Tampilkan frame (opsional, hanya untuk tujuan visualisasi)
        cv2.imshow('Frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Tutup video
    cap.release()
    cv2.destroyAllWindows()

    # Gabungkan nilai EAR dari video saat ini ke dalam list utama
    ear_values_all.extend(ear_values)

# Hitung statistik EAR
mean_ear = np.mean(ear_values_all)
min_ear = np.min(ear_values_all)
max_ear = np.max(ear_values_all)

# Plot grafik EAR
frame_indices = np.arange(1, len(ear_values_all) + 1)
plt.plot(frame_indices, ear_values_all)
plt.axhline(y=mean_ear, color='r', linestyle='--', label=f'Rata-rata: {mean_ear:.2f}')
plt.axhline(y=min_ear, color='g', linestyle='--', label=f'Minimum: {min_ear:.2f}')
plt.axhline(y=max_ear, color='b', linestyle='--', label=f'Maksimum: {max_ear:.2f}')
plt.title('Grafik Eye Aspect Ratio (EAR) terhadap Waktu')
plt.xlabel('Nomor Frame')
plt.ylabel('EAR')
plt.legend()
plt.grid(True)
plt.show()


RuntimeError: Unsupported image type, must be 8bit gray or RGB image.

## Percobaan Nilai EAR

Pada kode dibawah ini, mengestrakan data dalam bentuk vidio menjadi gambar berdasarkan nilai ear.
dengan tujuan untuk mencari nilai ear yang sesuai. pada bagian ini akan di coba beberapa nilai ear

In [18]:
import cv2
import dlib
import os
import numpy as np
from imutils import face_utils
from scipy.spatial import distance as dist



def calculate_eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    EAR = (A + B) / (2.0 * C)
    return EAR

(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_68_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_68_IDXS["right_eye"]


detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
eye_aspect_ratio_threshold = 0.38


counter = 0

# Menyimpan path folder video
video_folder = "D:\TA-main\data"

# Looping semua file video dalam folder
for filename in os.listdir(video_folder):
    if filename.endswith(".avi"):
        video_path = os.path.join(video_folder, filename)
        cap = cv2.VideoCapture(video_path)

        while True:
            ret, frame = cap.read()
            if not ret:
                print(f"Video {filename} telah selesai diputar.")
                break

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = detector(gray)
                
            for face in faces:
                shape = predictor(gray, face)
                shape = face_utils.shape_to_np(shape)
                left_eye = shape[lStart:lEnd]
                right_eye = shape[rStart:rEnd]
                leftEAR = calculate_eye_aspect_ratio(left_eye)
                rightEAR = calculate_eye_aspect_ratio(right_eye)
                EAR = (leftEAR + rightEAR) / 2.0
           
                
                #menggambarkan daerah mata
                cv2.polylines(frame, [np.array(left_eye)], True, (0, 0, 255), 1)
                cv2.polylines(frame, [np.array(right_eye)], True, (0, 0, 255), 1)
                cv2.putText(frame, "EAR: {:.2f}".format(EAR), (150, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
           

                if EAR > eye_aspect_ratio_threshold:
                    output_folder = "tidak mengantuk"
                elif EAR < eye_aspect_ratio_threshold:
                    output_folder = "mengantuk"
                
                cv2.imshow("Face Detection", frame)
            

                os.makedirs(output_folder, exist_ok=True)
                output_path = os.path.join(output_folder, f"{filename}_frame_{counter}.jpg")
                cv2.imwrite(output_path, frame)
                counter += 1

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()

cv2.destroyAllWindows()


RuntimeError: Unsupported image type, must be 8bit gray or RGB image.

# Analisis MAR 

Pada bagian ini divisisualisakan nilai mar dengan tujuan untuk melihat dimana nilai mar yang
menunjukkan kondisi mulut sedang tertutup atau terbuka.

In [20]:
import cv2
import numpy as np
import dlib
import matplotlib.pyplot as plt
import os

# Fungsi untuk menghitung Mouth Aspect Ratio (MAR)
def mouth_aspect_ratio(mouth):
    A = np.linalg.norm(mouth[3] - mouth[9])
    B = np.linalg.norm(mouth[0] - mouth[6])
    mar = A / B
    return mar

# Fungsi untuk mendeteksi wajah dan landmark
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Folder tempat video disimpan
video_folder = "D:\TA-main\data"

# Daftar video dalam folder
video_files = [f for f in os.listdir(video_folder) if f.endswith('.avi')]

mar_values_all = []  # List untuk menyimpan nilai MAR dari semua video

for video_file in video_files:
    video_path = os.path.join(video_folder, video_file)
    cap = cv2.VideoCapture(video_path)

    mar_values = []  # List untuk menyimpan nilai MAR dari setiap frame

    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # Ubah ke grayscale untuk deteksi
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Deteksi wajah
        faces = detector(gray)

        for face in faces:
            landmarks = predictor(gray, face)

            # Ambil koordinat mulut
            mouth = []
            for n in range(48, 68):
                x = landmarks.part(n).x
                y = landmarks.part(n).y
                mouth.append((x, y))

            # Hitung MAR untuk mulut
            mar = mouth_aspect_ratio(np.array(mouth))
            mar_values.append(mar)
            
            # Gambar garis di sekitar mulut (opsional, hanya untuk tujuan visualisasi)
            cv2.polylines(frame, [np.array(mouth)], True, (0, 255, 0), 1)
            
        # Tampilkan frame (opsional, hanya untuk tujuan visualisasi)
        cv2.imshow('Frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Tutup video
    cap.release()
    cv2.destroyAllWindows()

    # Gabungkan nilai MAR dari video saat ini ke dalam list utama
    mar_values_all.extend(mar_values)

# Hitung statistik MAR
mean_mar = np.mean(mar_values_all)
min_mar = np.min(mar_values_all)
max_mar = np.max(mar_values_all)

# Plot grafik MAR
frame_indices = np.arange(1, len(mar_values_all) + 1)
plt.plot(frame_indices, mar_values_all)
plt.axhline(y=mean_mar, color='r', linestyle='--', label=f'Rata-rata: {mean_mar:.2f}')
plt.axhline(y=min_mar, color='g', linestyle='--', label=f'Minimum: {min_mar:.2f}')
plt.axhline(y=max_mar, color='b', linestyle='--', label=f'Maksimum: {max_mar:.2f}')
plt.title('Grafik Mouth Aspect Ratio (MAR) terhadap Waktu')
plt.xlabel('Nomor Frame')
plt.ylabel('MAR')
plt.legend()
plt.grid(True)
plt.show()


RuntimeError: Unsupported image type, must be 8bit gray or RGB image.

## Percobaan Nilai MAR

Pada kode dibawah ini, mengestrakan data dalam bentuk vidio menjadi gambar berdasarkan nilai mar.
dengan tujuan untuk mencari nilai mar yang sesuai. pada bagian ini akan di coba beberapa nilai mar

In [21]:
import cv2
import dlib
import os
import numpy as np
from imutils import face_utils
from scipy.spatial import distance as dist


def calculate_lip_aspect_ratio(lips):
    dist1 = dist.euclidean(lips[3], lips[9]) 
    dist2 = dist.euclidean(lips[0], lips[6]) 
    lar = float(dist1/dist2)
    return lar

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
lip_aspect_ratio_threshold = 0.40


counter = 0

# Menyimpan path folder video
video_folder = "D:\TA-main\data"

# Looping semua file video dalam folder
for filename in os.listdir(video_folder):
    if filename.endswith(".avi"):
        video_path = os.path.join(video_folder, filename)
        cap = cv2.VideoCapture(video_path)

        while True:
            ret, frame = cap.read()
            if not ret:
                print(f"Video {filename} telah selesai diputar.")
                break

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = detector(gray)
            for face in faces:
                shape = predictor(gray, face)
                shape = face_utils.shape_to_np(shape)
                lips = shape[48:68]
                lar = calculate_lip_aspect_ratio(lips)
                
                
           

                if lar > lip_aspect_ratio_threshold:
                    output_folder = "menguap"
                elif lar < lip_aspect_ratio_threshold:
                    output_folder = "tidak menguap"
                
                cv2.imshow("Face Detection", frame)
            

                os.makedirs(output_folder, exist_ok=True)
                output_path = os.path.join(output_folder, f"{filename}_frame_{counter}.jpg")
                cv2.imwrite(output_path, frame)
                counter += 1

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()

cv2.destroyAllWindows()


RuntimeError: Unsupported image type, must be 8bit gray or RGB image.

# Ektraksi Data

Setelah nilai terbaik dari ear dan mar, selanjutnya pada bagian ini data video akan di ekstrak menjadi gambar
berdasarkan kedua parameter tersebut menjadi 4 kelas, yang merupakan kombinasi dari mar dan ear.

In [22]:
import cv2
import dlib
import os
import numpy as np
from imutils import face_utils
from scipy.spatial import distance as dist

def calculate_eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    EAR = (A + B) / (2.0 * C)
    return EAR

(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_68_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_68_IDXS["right_eye"]

def calculate_lip_aspect_ratio(lips):
    dist1 = dist.euclidean(lips[3], lips[9]) 
    dist2 = dist.euclidean(lips[0], lips[6]) 
    lar = float(dist1/dist2)
    return lar

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
eye_aspect_ratio_threshold = 0.20  
lip_aspect_ratio_threshold = 0.50


counter = 0

# Menyimpan path folder video
video_folder = "D:\TA-main\data"

# Looping semua file video dalam folder
for filename in os.listdir(video_folder):
    if filename.endswith(".avi"):
        video_path = os.path.join(video_folder, filename)
        cap = cv2.VideoCapture(video_path)

        while True:
            ret, frame = cap.read()
            if not ret:
                print(f"Video {filename} telah selesai diputar.")
                break

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = detector(gray)
            for face in faces:
                shape = predictor(gray, face)
                shape = face_utils.shape_to_np(shape)
                left_eye = shape[lStart:lEnd]
                right_eye = shape[rStart:rEnd]
                lips = shape[48:68]
                leftEAR = calculate_eye_aspect_ratio(left_eye)
                rightEAR = calculate_eye_aspect_ratio(right_eye)
                EAR = (leftEAR + rightEAR) / 2.0
                lar = calculate_lip_aspect_ratio(lips)
                
                # menggambarkan daerah bibir
#                 cv2.putText(frame, "LAR: {:.2f}".format(lar), (315, 30),
#                 cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
#                 cv2.polylines(frame, [np.array(lips)], True, (0, 255, 0), 1)
                
#                 #menggambarkan daerah mata
#                 cv2.polylines(frame, [np.array(left_eye)], True, (0, 0, 255), 1)
#                 cv2.polylines(frame, [np.array(right_eye)], True, (0, 0, 255), 1)
#                 cv2.putText(frame, "EAR: {:.2f}".format(EAR), (150, 30),
#                 cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                
                
           

                if EAR < eye_aspect_ratio_threshold and lar > lip_aspect_ratio_threshold:
                    output_folder = "mengantuk & menguap"
                elif EAR < eye_aspect_ratio_threshold and lar < lip_aspect_ratio_threshold:
                    output_folder = "mengantuk & tidak menguap"
                elif EAR > eye_aspect_ratio_threshold and lar > lip_aspect_ratio_threshold:
                    output_folder = "tidak mengantuk & menguap"
                elif EAR > eye_aspect_ratio_threshold and lar < lip_aspect_ratio_threshold:
                    output_folder = "tidak mengantuk & tidak menguap"
                
                cv2.imshow("Face Detection", frame)
            

                os.makedirs(output_folder, exist_ok=True)
                output_path = os.path.join(output_folder, f"{filename}_frame_{counter}.jpg")
                cv2.imwrite(output_path, frame)
                counter += 1

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()

cv2.destroyAllWindows()


RuntimeError: Unsupported image type, must be 8bit gray or RGB image.